# Project Group - 5

Members: Ahmad Nabil Maulana, Daan Michel, Gijs Aben, Philine Cremers

Student numbers: 5943442, 4684559 ,    4713656   ,     5036534

# Research Objective

*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

The research objective is to analyse the Impact of Gross Domestic Product pet capita (GDP) on the number of air traffic movements within a country. 
**(We would like to have some feedback on our research objective if is defined enough or specific enough. Furthermore we would like to know if you foresee any problems that we could have with the data packages that we're going to use, especially for the air traffic data. At last we would like to hear your suggestions regarding data visualization, because we think this project is really straightforward, so we like some suggestions on how we're going to handle this project)**

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**: Ahmad Nabil Maulana : Creating the repository,

**Author 2**: Daan Michel

**Author 3**: Philine Cremers

**Author 4**: Gijs Aben

# Background and Context

For this project, we will analyze the relationship between a country's GDP per capita and its air traffic mobility. GDP measures the monetary value of final goods and services produced within a country's borders during a specific period, such as a quarter or a year. We aim to determine whether countries with higher GDPs exhibit greater air traffic mobility compared to those with lower GDPs, or if there is no discernible correlation

For the flight data we found two different data sets with each its own pros and cons. \
The different data sets form OpenSky provide information about all the aircraft movements. We can use all air traffic movements to and from all the airports in a country. The data will be filtered by only picking comercial aircraft and cargo aircraft, private aircraft will not be considered. Furthermore the origin country of the departed flights will be used, which gives us an overview of amount of takeoffs per country. This should be a good indicator for the relative amount of air traffic movements per country when compared to the GDP. However the amount of data that needs to be processed for this is huge and could potentially lead to problems.\
The second data set we found is from The World Bank: 'Air transport, registered carrier departures worldwide'. This data discribes the amount of registerd aircraft takeoffs on scheduled services (i.e. plannend commercial flights both passenger and cargo) by country the aircraft is registered in. The data set gives us takeoffs per country per year which should make processing the data not too complicated and thus make it possible to go for an overview of multiple years. However, this data might not completely represent the goals of our research objective since it does not discripe the amount of aircraft movements per country. An aircraft can be registered in a country where it does not depart from or arrive in.

As the first step, we need to import the necessary libraries.

In [3]:
# this can be changed
import pandas as pd
from pathlib import Path
import numpy as np
import math
import scipy
from scipy.signal import find_peaks
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import os

# Part I - Data Import

First, we're going to import and combine dataframes from the two datas that we will be using:

* Countries' GDP data from The World Bank.
* Countries' air traffic mobility data from OpenSky or The World Bank.

In [4]:
# import the file for the GDP data
file_path_GDP = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/Data/GDP-US/GDP_per_state_all_states.csv'
df_GDP = pd.read_csv(file_path_GDP)

df_GDP.head()

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2005:Q1,2005:Q2,...,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1,2022:Q2,2022:Q3,2022:Q4,2023:Q1
0,"""01000""",Alabama,5,SQGDP9,1,...,All industry total,Millions of chained 2012 dollars,182600.4,184625.7,...,204400.7,207001.8,209857.1,210029.1,213029.2,212789.5,212311.6,212946.3,215011.6,215084.4
1,"""02000""",Alaska,8,SQGDP9,1,...,All industry total,Millions of chained 2012 dollars,45176.1,45776.9,...,51069.4,50690.3,50707.9,50935.9,51143.5,49070.4,48963.4,49999.1,50501.7,50700.8
2,"""04000""",Arizona,6,SQGDP9,1,...,All industry total,Millions of chained 2012 dollars,256521.4,260884.6,...,338182.1,339659.7,344936.2,348706.0,357322.0,355309.3,353565.6,356966.0,359827.3,362191.9
3,"""05000""",Arkansas,5,SQGDP9,1,...,All industry total,Millions of chained 2012 dollars,104693.3,105271.5,...,119689.9,121860.2,123015.2,123676.9,124837.0,126803.5,125830.9,126248.3,127245.8,127312.2
4,"""06000""",California,8,SQGDP9,1,...,All industry total,Millions of chained 2012 dollars,1896451.4,1913819.0,...,2746311.9,2802569.5,2858504.7,2894880.5,2942968.5,2870410.5,2866766.2,2893948.3,2911384.3,2919913.3


In [8]:
# import the file for the air mobility data
file_path_registered_carrier = 'https://raw.githubusercontent.com/anmaulana1/big-project/main/air_mobility_worldbank.csv'
df_air_mobility = pd.read_csv(file_path_registered_carrier, on_bad_lines='skip')
df_air_mobility.head(15)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,ABW,"Air transport, registered carrier departures w...",IS.AIR.DPRT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2132.00,2.276000e+03,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,"Air transport, registered carrier departures w...",IS.AIR.DPRT,NaN,NaN,NaN,NaN,NaN,NaN,...,5.488348e+05,5.348103e+05,556341.0,562927.0,630147.00,7.051275e+05,7.177953e+05,286064.1974,399895.848,NaN
2,Afghanistan,AFG,"Air transport, registered carrier departures w...",IS.AIR.DPRT,NaN,NaN,NaN,NaN,NaN,NaN,...,2.169600e+04,2.592000e+04,23532.0,22770.0,24207.00,1.045400e+04,7.334000e+03,4635.7140,2865.737,NaN
3,Africa Western and Central,AFW,"Air transport, registered carrier departures w...",IS.AIR.DPRT,NaN,NaN,NaN,NaN,NaN,NaN,...,1.550380e+05,1.455268e+05,164614.0,157788.0,151203.00,1.571267e+05,1.588748e+05,92611.3490,134532.013,NaN
4,Angola,AGO,"Air transport, registered carrier departures w...",IS.AIR.DPRT,NaN,NaN,NaN,NaN,NaN,NaN,...,1.449600e+04,1.371600e+04,13116.0,15482.0,13494.00,1.397800e+04,1.364700e+04,3792.0000,3805.000,NaN
5,Albania,ALB,"Air transport, registered carrier departures w...",IS.AIR.DPRT,NaN,NaN,NaN,NaN,NaN,NaN,...,1.119600e+04,1.992000e+03,NaN,306.0,1904.00,2.935000e+03,2.558000e+03,1274.0000,1471.000,NaN
6,Andorra,AND,"Air transport, registered carrier departures w...",IS.AIR.DPRT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Arab World,ARB,"Air transport, registered carrier departures w...",IS.AIR.DPRT,NaN,NaN,NaN,NaN,NaN,NaN,...,1.222184e+06,1.289639e+06,1375862.0,1476812.0,1503553.38,1.539593e+06,1.599362e+06,664879.5750,920600.238,NaN
8,United Arab Emirates,ARE,"Air transport, registered carrier departures w...",IS.AIR.DPRT,NaN,NaN,NaN,NaN,NaN,NaN,...,3.270760e+05,3.522240e+05,437638.0,463947.0,459137.00,4.559560e+05,4.261573e+05,185260.0000,234612.000,NaN
9,Argentina,ARG,"Air transport, registered carrier departures w...",IS.AIR.DPRT,NaN,NaN,NaN,NaN,NaN,NaN,...,1.451360e+05,1.345210e+05,145585.0,149334.0,146631.00,1.618620e+05,1.631060e+05,27447.0000,54218.000,NaN


# Part II - Data Processing